In [3]:
import numpy as np
import pandas as pd
import altair as alt

# Save a vega-lite spec and a PNG blob for each plot in the notebook
alt.renderers.enable("mimetype")
# Handle large data sets without embedding them in the notebook
alt.data_transformers.enable("data_server");

In [4]:
stinson2019_df = pd.read_pickle("../data/stinson2019/stinson2019_df")
HF_weather = pd.read_csv('../data/HF_weather/raw/hf001-08-hourly-m.csv', parse_dates=['datetime'])

/home/steffen/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3,5,7,9,11,13,15,17,19,21,23,25,27,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
stinson2019_df["jd"] = pd.DatetimeIndex(stinson2019_df["date"]).dayofyear
stinson2019_df["year_int"] = pd.DatetimeIndex(stinson2019_df["year"]).year
stinson2019_df["sugar_wt"] = stinson2019_df["sugar"]/100 * stinson2019_df["sap_wt"]
stinson2019_df["yearly_sap"] = np.nan
stinson2019_df["yearly_sugar"] = np.nan
stinson2019_df['tree_tap'] = stinson2019_df['tree']+ '_' + stinson2019_df['tap']
stinson2019_df = stinson2019_df.sort_values(by="date")

abs_measurement = 0

for year in stinson2019_df["year_int"].unique():
    year_df = stinson2019_df[stinson2019_df["year_int"] == year]

    for site in stinson2019_df[stinson2019_df["year_int"] == year]["site"].unique():
        for tree in stinson2019_df[
            (stinson2019_df["year_int"] == year) & (stinson2019_df["site"] == site)
        ]["tree"].unique():
            for tap in stinson2019_df[
                (stinson2019_df["year_int"] == year)
                & (stinson2019_df["site"] == site)
                & (stinson2019_df["tree"] == tree)
            ]["tap"].unique():
                tap_yr_index = stinson2019_df.index[
                    (stinson2019_df["year_int"] == year)
                    & (stinson2019_df["site"] == site)
                    & (stinson2019_df["tree"] == tree)
                    & (stinson2019_df["tap"] == tap)
                ].tolist()
                yearly_sap = 0
                yearly_sugar = 0
                for measurement in tap_yr_index:
                    yearly_sap = np.nansum(
                        [yearly_sap, stinson2019_df.loc[measurement, "sap_wt"]]
                    )
                    yearly_sugar = np.nansum(
                        [
                            yearly_sugar,
                            stinson2019_df.loc[measurement, "sap_wt"]
                            * stinson2019_df.loc[measurement, "sugar"]/100,
                        ]
                    )
                    stinson2019_df.loc[measurement, "yearly_sap"] = yearly_sap
                    stinson2019_df.loc[measurement, "yearly_sugar"] = yearly_sugar


#     yearly_sap = 0
#     yearly_sugar = 0
#     for measurement in list(range(len(year_df))):
#         yearly_sap+=stinson2019_df.iloc[abs_measurement,6]
#         yearly_sugar+=stinson2019_df.iloc[abs_measurement,6]*stinson2019_df.iloc[abs_measurement,5]
#         stinson2019_df.iloc[abs_measurement,13]=yearly_sap
#         stinson2019_df.iloc[abs_measurement,14]=yearly_sugar
#         abs_measurement+=1

In [6]:
stinson2019_df

,site,date,tree,species,tap,sugar,sap_wt,year,site_id,tree_id,tree_record_id,tap_record_id,jd,year_int,sugar_wt,yearly_sap,yearly_sugar,tree_tap
0,Harvard Forest,2012-02-16,HF1,ACSA,A,2.7,4.24,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-16,HF.ACSA.HF1.A.2012-02-16,47,2012,0.11448,4.24,0.11448,HF1_A
22,Harvard Forest,2012-02-16,HF43,ACSA,B,3.1,4.16,2012-01-01,HF,HF.ACSA.HF43,HF.ACSA.HF43.2012-02-16,HF.ACSA.HF43.B.2012-02-16,47,2012,0.12896,4.16,0.12896,HF43_B
21,Harvard Forest,2012-02-16,HF43,ACSA,A,3.2,8.42,2012-01-01,HF,HF.ACSA.HF43,HF.ACSA.HF43.2012-02-16,HF.ACSA.HF43.A.2012-02-16,47,2012,0.26944,8.42,0.26944,HF43_A
20,Harvard Forest,2012-02-16,HF41,ACSA,B,3.8,1.62,2012-01-01,HF,HF.ACSA.HF41,HF.ACSA.HF41.2012-02-16,HF.ACSA.HF41.B.2012-02-16,47,2012,0.06156,1.62,0.06156,HF41_B
19,Harvard Forest,2012-02-16,HF41,ACSA,A,3.7,2.36,2012-01-01,HF,HF.ACSA.HF41,HF.ACSA.HF41.2012-02-16,HF.ACSA.HF41.A.2012-02-16,47,2012,0.08732,2.36,0.08732,HF41_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9092,Northern range - Quebec,2017-05-04,QC3,ACSA,A,NaN,0.50,2017-01-01,QC,QC.ACSA.QC3,QC.ACSA.QC3.2017-05-04,QC.ACSA.QC3.A.2017-05-04,124,2017,NaN,34.11,0.86284,QC3_A
9091,Northern range - Quebec,2017-05-04,QC2,ACSA,A,1.6,0.11,2017-01-01,QC,QC.ACSA.QC2,QC.ACSA.QC2.2017-05-04,QC.ACSA.QC2.A.2017-05-04,124,2017,0.00176,16.99,0.31621,QC2_A
9090,Northern range - Quebec,2017-05-04,QC1,ACSA,A,1.7,1.18,2017-01-01,QC,QC.ACSA.QC1,QC.ACSA.QC1.2017-05-04,QC.ACSA.QC1.A.2017-05-04,124,2017,0.02006,18.18,0.43791,QC1_A
9105,Northern range - Quebec,2017-05-04,QC16,ACSA,A,1.6,0.34,2017-01-01,QC,QC.ACSA.QC16,QC.ACSA.QC16.2017-05-04,QC.ACSA.QC16.A.2017-05-04,124,2017,0.00544,15.93,0.33106,QC16_A


In [7]:
harvard_sap = stinson2019_df[stinson2019_df['site_id']=='HF']
harvard_sap['jd'].max()

102

In [8]:
weeks = list(range(0,106,7))
harvard_sap.groupby([harvard_sap['year'], pd.cut(harvard_sap['jd'], weeks)]).sum()

sugar  sap_wt       jd  year_int  sugar_wt  yearly_sap  \
year       jd                                                                  
2012-01-01 (0, 7]       NaN     NaN      NaN       NaN       NaN         NaN   
           (7, 14]      NaN     NaN      NaN       NaN       NaN         NaN   
           (14, 21]     NaN     NaN      NaN       NaN       NaN         NaN   
           (21, 28]     NaN     NaN      NaN       NaN       NaN         NaN   
           (28, 35]     NaN     NaN      NaN       NaN       NaN         NaN   
...                     ...     ...      ...       ...       ...         ...   
2017-01-01 (70, 77]     NaN     NaN      NaN       NaN       NaN         NaN   
           (77, 84]   196.8  287.54   7632.0  193632.0   6.73874     4241.60   
           (84, 91]   498.2  823.22  20880.0  484080.0  18.96144    13784.15   
           (91, 98]   243.8  343.14  12300.0  266244.0   7.71448     9610.41   
           (98, 105]    NaN     NaN      NaN       NaN       NaN         NaN   

                      yearly_sugar  
year       jd                       
2012-01-01 (0, 7]              NaN  
           (7, 14]             NaN  
           (14, 21]            NaN  
           (21, 28]            NaN  
           (28, 35]            NaN  
...                            ...  
2017-01-01 (70, 77]            NaN  
           (77, 84]      103.54440  
           (84, 91]      332.57985  
           (91, 98]      231.82877  
           (98, 105]           NaN  

[90 rows x 7 columns]

In [9]:
pd.periodindex

AttributeError: module 'pandas' has no attribute 'periodindex'